In [1]:
import random
import pandas as pd
import Covid19_model as CM
import numpy as np
import shutil
import sys
import os.path
import math
import csv
import lhsmdu
import time
from Pyomo_solve import solve_ode

# Set parameter

In [2]:
D = 1e2
n_site = 4
pop = np.array(pd.read_csv("population_"+str(n_site)+".csv",header=None))
pop = np.reshape(np.ones((6, 1)) * pop, (6, n_site))
init_state = np.array(pd.read_csv("state_"+str(n_site)+".csv",header=None))
density = np.array(pd.read_csv("density_"+str(n_site)+".csv",header=None))
dij = np.array(pd.read_csv("distance_"+str(n_site)+".csv",header=None)) / D
N = np.sum(init_state)
n_state = 7 * n_site + 1
n_action = 5 * n_site
density = np.reshape(density, (1, n_site))
T = 90
budget = 1e6
length = 50

In [3]:
deter_para = dict(N=N, P=pop, B=budget, T=T, D=dij, site=n_site, density=density,
                  gamma=0.69, alpha=0.6, v_max = 0.2/14,
                  cost_ti=0.0977, cost_ta=0.02, cost_v=0.07, 
                  cost_poc_0=0.000369, cost_poc_1=0.001057, 
                  pid=1.10/1000, psr=0.7/3, pid_plus=0.1221/1000, pir=1/8, prs=1/180)

in_deter_para = dict(beta2=[0.15204, 0.16287], beta1=[0.7602, 0.81435],
                     pei=[0.07143, 0.14286], per=[0.04000,0.05556])

in_deter_truth = dict(beta2=0.15747, beta1=0.78735, pei=0.10714, per=0.04545)

para_truth = {**deter_para, **in_deter_truth}

In [4]:
seed = 0
env = CM.Env_model(init_state, deter_para, in_deter_para, seed)
env.set_para_truth(para_truth)

In [5]:
tic = time.time()
random.seed(seed)
observation, state = init_state.copy(), init_state.copy()
state_buffer, new_buffer  = np.zeros([T, 6]), np.zeros([T, 4])
score, ni, ne = 0, 0, 0
done = False
index = [0]
start_time = 6
#n_select = 10
for t in range(T):
    p_set = env.para_sampling(100)
    para = p_set[index[0]]
    action_buffer = [] 
    reward_buffer = []
    s = np.array(state)
    ac = []
    if t > start_time:
        for a in range(n_site):
            try:
                aa = solve_ode(s[:,a], np.sum(s[1:-1,a])*budget/np.sum(s[1:-1,:]), para, length)
            except Exception:
                print("pyomo fail")
            ac.append(aa)
        action = np.array(ac).T
        if budget < 0: 
            action = np.zeros((5,n_site))
    else:
        action = np.zeros((5,n_site))
        
    
    next_state, obs, reward, cost, new_i, new_e, ck = env.online_state_estimate(state, observation, 
                                                                                action, p_set, 100, 1)
    #if not done and t > start_time: 
     #   env.space_update(ck, p_set, n_select)
    index = ck.argsort()[0][:1]
    score += reward
    budget -= cost
    ni += new_i
    ne += new_e

    new_buffer[t, 0], new_buffer[t, 1], new_buffer[t, 2], new_buffer[t, 3] = new_i, ni, new_e, ne
    print('\r', 'True State Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}, Budget: {:.2f}, score:{:.2f}'
          .format(t, np.sum(observation[0]), np.sum(observation[1]), np.sum(observation[2]), 
                  np.sum(observation[3]), np.sum(observation[4]), np.sum(observation[5]),
                  budget, score))
    print('\r', 'Observe Time: {} | S: {:.1f}, E: {:.1f}, A: {:.1f}, I: {:.1f}, D: {:.1f}, R: {:.1f}'
          .format(t, np.sum(state[0]), np.sum(state[1]), np.sum(state[2]), np.sum(state[3]),
                  np.sum(state[4]), np.sum(state[5]))) 
    print('\r', 'Time: {} | new_e:{:.1f}, cum_e:{:.1f}, new_i:{:.1f}, cum_i:{:.1f}'.format(t, new_e, ne, new_i, ni))
    state_buffer[t, :] = sum(observation.T)
    state = next_state.copy()
    observation = obs.copy()
print(time.time() - tic)

 True State Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0, Budget: 1000000.00, score:-57.32
 Observe Time: 0 | S: 12563972.0, E: 28.0, A: 0.0, I: 0.0, D: 0.0, R: 0.0
 Time: 0 | new_e:13.2, cum_e:13.2, new_i:11.8, cum_i:11.8
 True State Time: 1 | S: 12563950.0, E: 37.0, A: 0.0, I: 11.8, D: 0.0, R: 1.3, Budget: 1000000.00, score:-138.59
 Observe Time: 1 | S: 12563949.0, E: 36.0, A: 0.0, I: 11.8, D: 0.0, R: 0.0
 Time: 1 | new_e:18.6, cum_e:31.8, new_i:16.3, cum_i:28.2
 True State Time: 2 | S: 12563919.0, E: 49.9, A: 0.0, I: 28.1, D: 0.0, R: 2.9, Budget: 1000000.00, score:-254.10
 Observe Time: 2 | S: 12563919.0, E: 50.0, A: 0.0, I: 28.1, D: 0.0, R: 0.0
 Time: 2 | new_e:26.2, cum_e:58.0, new_i:22.8, cum_i:51.0
 True State Time: 3 | S: 12563875.3, E: 68.5, A: 0.0, I: 50.9, D: 0.0, R: 5.2, Budget: 1000000.00, score:-418.56
 Observe Time: 3 | S: 12563877.0, E: 68.0, A: 0.0, I: 50.9, D: 0.0, R: 3.0
 Time: 3 | new_e:37.2, cum_e:95.2, new_i:32.1, cum_i:83.1
 True State Time: 4

 True State Time: 21 | S: 12053380.4, E: 10.2, A: 112.6, I: 292.0, D: 1.4, R: 510203.3, Budget: 611909.70, score:-3345.91
 Observe Time: 21 | S: 12053422.0, E: 9.0, A: 112.6, I: 292.0, D: 1.4, R: 510199.0
 Time: 21 | new_e:26.4, cum_e:750.7, new_i:30.7, cum_i:955.7
pyomo fail
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 22 | S: 12031752.8, E: 29.3, A: 101.1, I: 296.3, D: 1.5, R: 531818.9, Budget: 587579.57, score:-3473.03
 Observe Time: 22 | S: 12031795.0, E: 29.0, A: 101.1, I: 296.3, D: 1.5, R: 531815.0
 Time: 22 | new_e:28.1, cum_e:778.8, new_i:32.7, cum_i:988.4
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
pyomo fail
 True State Time: 23 | S: 12000623.7, E: 32.1, A: 106.5, I: 299.0

 True State Time: 43 | S: 11633723.5, E: 9.9, A: 13.6, I: 72.3, D: 2.3, R: 930178.5, Budget: 315885.57, score:-3855.30
 Observe Time: 43 | S: 11633807.0, E: 0.0, A: 13.6, I: 72.3, D: 2.3, R: 930173.0
 Time: 43 | new_e:4.3, cum_e:857.8, new_i:5.4, cum_i:1176.9
 True State Time: 44 | S: 11616160.6, E: 11.6, A: 12.5, I: 68.6, D: 2.3, R: 947744.4, Budget: 306275.39, score:-3877.55
 Observe Time: 44 | S: 11616252.0, E: 0.0, A: 12.5, I: 68.6, D: 2.3, R: 947738.0
 Time: 44 | new_e:5.0, cum_e:862.8, new_i:5.9, cum_i:1182.8
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 45 | S: 11594629.8, E: 13.6, A: 11.9, I: 66.0, D: 2.3, R: 969276.4, Budget: 302083.84, score:-3915.82
 Observe Time: 45 | S: 11594729.0, E: 0.0, A: 11.9, I: 66.0, D: 2.3, R: 969270.0
 Time: 45 | new_e:8.5, cum_e:871.3, new_i:8.4, cum_i:1191.2
 True State Time: 46 | S: 11

 True State Time: 64 | S: 11485570.8, E: 1.8, A: 29.1, I: 67.6, D: 2.6, R: 1078328.1, Budget: 83541.72, score:-4454.64
 Observe Time: 64 | S: 11485804.0, E: 0.0, A: 29.1, I: 67.6, D: 2.6, R: 1078316.0
 Time: 64 | new_e:0.8, cum_e:992.0, new_i:3.8, cum_i:1358.5
 True State Time: 65 | S: 11480622.8, E: 2.2, A: 24.7, I: 63.0, D: 2.6, R: 1083284.6, Budget: 78939.52, score:-4459.22
 Observe Time: 65 | S: 11480857.0, E: 0.0, A: 24.7, I: 63.0, D: 2.6, R: 1083272.0
 Time: 65 | new_e:1.0, cum_e:992.9, new_i:3.5, cum_i:1362.1
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 66 | S: 11473108.9, E: 2.8, A: 21.0, I: 58.6, D: 2.6, R: 1090806.0, Budget: 74701.44, score:-4480.47
 Observe Time: 66 | S: 11473344.0, E: 0.0, A: 21.0, I: 58.6, D: 2.6, R: 1090795.0
 Time: 66 | new_e:4.6, cum_e:997.5, new_i:5.6, cum_i:1367.7
 True State Time: 67 | S: 1

 True State Time: 87 | S: 11513031.8, E: 207.3, A: 5.6, I: 372.1, D: 4.8, R: 1050378.4, Budget: 1775.52, score:-6565.27
 Observe Time: 87 | S: 11513410.0, E: 168.0, A: 5.6, I: 372.1, D: 4.8, R: 1050366.0
 Time: 87 | new_e:88.6, cum_e:1443.1, new_i:81.9, cum_i:1806.1
    model.name="unknown";
      - termination condition: infeasible
      - message from solver: Ipopt 3.14.6\x3a Converged to a locally
        infeasible point. Problem may be infeasible.
 True State Time: 88 | S: 11518719.6, E: 264.2, A: 4.8, I: 453.6, D: 5.2, R: 1044552.7, Budget: 1769.11, score:-7139.08
 Observe Time: 88 | S: 11519128.0, E: 207.0, A: 4.8, I: 453.6, D: 5.2, R: 1044539.0
 Time: 88 | new_e:125.5, cum_e:1568.6, new_i:112.5, cum_i:1918.6
 True State Time: 89 | S: 11524313.4, E: 349.4, A: 4.0, I: 565.6, D: 5.7, R: 1038761.8, Budget: 1757.72, score:-7824.47
 Observe Time: 89 | S: 11524767.0, E: 269.0, A: 4.0, I: 565.6, D: 5.7, R: 1038745.0
 Time: 89 | new_e:149.6, cum_e:1718.2, new_i:137.6, cum_i:2056.2
3842.

In [1]:
3842.85253906 / ( T - start_time + 1)

45.21002987129412